# 네이버 블로그 크롤링 Selenium + Beautifulsoup

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import requests
from bs4 import BeautifulSoup
import time


import time
import os

import pandas as pd
import re

In [7]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [12]:
def NaberCrawling (keyword : str, browser : str):
    keyword_input = keyword
    keyword_input = keyword_input.replace(' ','+')

    # browser = webdriver.Chrome(service=service,options=chrome_options)
    # browser.maximize_window()
    url = f"https://search.naver.com/search.naver?query={keyword_input}&nso=&where=blog&sm=tab_opt"
    browser.get(url)
    browser.implicitly_wait(2)


    for i in range(30):
        browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    links = browser.find_elements(By.CSS_SELECTOR,"div.total_area > a")
    blog_url = []
    for link in links:
        if ('blog.naver.com' in link.get_attribute('href')) and (link.get_attribute('href').find('b')==8):
            blog_url.append(link.get_attribute('href'))

    crwl_blog_title = []
    crwl_blog_text = []
    for i in range(len(blog_url)):
        blog_text = ''
        blog_title = ''
        browser.get(blog_url[i])
        browser.implicitly_wait(2.5)
        browser.switch_to.frame("mainFrame")
        soup = BeautifulSoup(browser.page_source,'html.parser')
        try:
            text=soup.select("div.se-module-text")
            title=soup.select("div.pcol1")
            time.sleep(0.5)
            for te in text:
                blog_text += te.text
            for ti in title:
                blog_title += ti.text
            crwl_blog_title.append(blog_title)
            crwl_blog_text.append(blog_text)
        except :
            print(i)

    return crwl_blog_title, crwl_blog_text

In [2]:
Keywords = ['istp특징', 'istj특징', 'isfp특징', 'isfj특징', 'intp특징', 'intj특징', 'infp특징', 'infj특징']

In [13]:
browser = webdriver.Chrome(service=service,options=chrome_options)
browser.maximize_window()
craw = []
for mbti in range(len(Keywords)):
    crwl = NaberCrawling(Keywords[mbti], browser)
    # 문자정리
    for i in range(2):
        for j in range(len(crwl[i])):
            crwl[i][j] = re.sub('\n+', '', crwl[i][j])
    #csv파일 만들기
    pd.DataFrame(crwl).to_csv(Keywords[mbti] + '네이버 블로그 크롤링.csv')

In [6]:
for mbti in range(len(Keywords)):
    df_mbti = []
    df_mbti = pd.read_csv('/Users/yerim/codes/test/'+Keywords[mbti] + '네이버 블로그 크롤링.csv')
    df_mbti = df_mbti.T
    pd.DataFrame(df_mbti).to_csv(Keywords[mbti]+'.csv')